# 10.Double-DQN

论文的作者建议使用训练网络来选择动作，但是使用目标网络的Q值。所以新的目标Q值为

$$
Q(s_t,a_t) = r_t+\gamma Q'(s_{t+1}, argmaxQ(s_{t+1}, a))
$$


In [1]:
import collections
import copy
import math
import random
import time
from collections import defaultdict

import gym
import gym.spaces
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from gym.envs.toy_text import frozen_lake
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, q_table_size):
        super(Net, self).__init__()

        self.net = nn.Sequential(
            # 输入为状态，样本为（1*n）
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            # nn.Linear(hidden_size, hidden_size),
            # nn.ReLU(),
            nn.Linear(hidden_size, q_table_size),
        )

    def forward(self, state):
        return self.net(state)


class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n,)
        self.observation_space = gym.spaces.Box(0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

# ReplayBuffer

In [3]:
class ReplayBuffer:
    def __init__(self, queue_size, replay_time):
        self.queue = []
        self.queue_size = queue_size
        self.replay_time = replay_time

    def get_batch_queue(self, env, action_trigger, batch_size, epsilon):
        def insert_sample_to_queue(env):
            state, info = env.reset()
            stop = 0

            while True:
                if np.random.uniform(0, 1, 1) > epsilon:
                    action = env.action_space.sample()
                else:
                    action = action_trigger(state)

                next_state, reward, terminated, truncated, info = env.step(action)
                self.queue.append([state, action, next_state, reward, terminated])
                state = next_state
                if terminated:
                    state, info = env.reset()
                    stop += 1
                    continue
                if stop >= replay_time:
                    break

        def init_queue(env):
            while True:
                insert_sample_to_queue(env)
                if len(self.queue) >= self.queue_size:
                    break

        init_queue(env)
        insert_sample_to_queue(env)
        self.queue = self.queue[-self.queue_size :]

        return random.sample(self.queue, batch_size)

# DQN

In [4]:
class DQN:
    def __init__(self, env, obs_size, hidden_size, q_table_size):
        self.env = env
        self.net = Net(obs_size, hidden_size, q_table_size)
        self.tgt_net = Net(obs_size, hidden_size, q_table_size)

    # 更新net参数
    def update_net_parameters(self, update=True):
        self.net.load_state_dict(self.tgt_net.state_dict())

    def get_action_trigger(self, state):
        state = torch.Tensor(state)
        action = int(torch.argmax(self.tgt_net(state).detach()))
        return action

    # 计算y_hat_and_y
    def calculate_y_hat_and_y(self, batch, gamma):
        y = []
        action_sapce = []
        state_sapce = []
        ## 
        for state, action, next_state, reward, terminated in batch:
            q_table_net = self.net(torch.Tensor(next_state)).detach()
            ## double DQN
            tgt_net_action = self.get_action_trigger(next_state)
            y.append(reward + (1 - terminated) * gamma * float(q_table_net[tgt_net_action]))
            action_sapce.append(action)
            state_sapce.append(state)
        y_hat = self.tgt_net(torch.Tensor(np.array(state_sapce)))
        y_hat = y_hat.gather(1, torch.LongTensor(action_sapce).reshape(-1, 1))
        return y_hat.reshape(-1), torch.tensor(y)

    def predict_reward(self):
        state, info = env.reset()
        step = 0
        reward_space = []

        while True:
            step += 1
            state = torch.Tensor(state)
            action = int(torch.argmax(self.net(state).detach()))
            next_state, reward, terminated, truncated, info = env.step(action)
            reward_space.append(reward)
            state = next_state
            if terminated:
                state, info = env.reset()
                continue
            if step >= 100:
                break
        return float(np.mean(reward_space))

## 训练

In [5]:
hidden_size = 64
queue_size = 500
replay_time = 50

## 初始化环境
env = frozen_lake.FrozenLakeEnv(is_slippery=False)
env.spec = gym.spec("FrozenLake-v1")
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)
env = DiscreteOneHotWrapper(env)

## 初始化buffer
replay_buffer = ReplayBuffer(queue_size, replay_time)

## 初始化dqn
obs_size = env.observation_space.shape[0]
q_table_size = env.action_space.n
dqn = DQN(env, obs_size, hidden_size, q_table_size)

# 定义优化器
opt = optim.Adam(dqn.tgt_net.parameters(), lr=0.01)

# 定义损失函数
loss = nn.MSELoss()

writer = SummaryWriter(log_dir="logs/DQN/Double_DQN_FrozenLake", comment="test1")

In [6]:
batch_size = 256
epsilon = 0.8
epochs = 500
gamma = 0.9

In [7]:
for epoch in range(epochs):
    batch = replay_buffer.get_batch_queue(
        env, dqn.get_action_trigger, batch_size, epsilon
    )
    y_hat, y = dqn.calculate_y_hat_and_y(batch, gamma)
    l = loss(y_hat, y)

    # 反向传播
    opt.zero_grad()
    l.backward()
    opt.step()

    if epoch % 10 == 0 and epoch != 0:
        dqn.update_net_parameters()

    predict_reward = dqn.predict_reward()
    writer.add_scalars(
        "MSE", {"loss": l.item(), "predict_reward": predict_reward}, epoch
    )

    print(
        "epoch:{},  MSE: {}, epsilon: {}, 100 steps reward: {}".format(
            epoch, l, epsilon, predict_reward
        )
    )

epoch:0,  MSE: 0.002034767996519804, epsilon: 0.8, 100 steps reward: 0.0
epoch:1,  MSE: 0.01553338672965765, epsilon: 0.8, 100 steps reward: 0.0
epoch:2,  MSE: 0.0041157943196594715, epsilon: 0.8, 100 steps reward: 0.0
epoch:3,  MSE: 0.005832789000123739, epsilon: 0.8, 100 steps reward: 0.0
epoch:4,  MSE: 0.004888113122433424, epsilon: 0.8, 100 steps reward: 0.0
epoch:5,  MSE: 0.009765115566551685, epsilon: 0.8, 100 steps reward: 0.0
epoch:6,  MSE: 0.0090354448184371, epsilon: 0.8, 100 steps reward: 0.0
epoch:7,  MSE: 0.007912073284387589, epsilon: 0.8, 100 steps reward: 0.0
epoch:8,  MSE: 0.005203092470765114, epsilon: 0.8, 100 steps reward: 0.0
epoch:9,  MSE: 0.004023554269224405, epsilon: 0.8, 100 steps reward: 0.0
epoch:10,  MSE: 0.004651330411434174, epsilon: 0.8, 100 steps reward: 0.0
epoch:11,  MSE: 0.004799548536539078, epsilon: 0.8, 100 steps reward: 0.0
epoch:12,  MSE: 0.004201455507427454, epsilon: 0.8, 100 steps reward: 0.0
epoch:13,  MSE: 0.003958794288337231, epsilon: 0.8

# 可视化预测

In [10]:
DQN_Q = dqn.net

env = frozen_lake.FrozenLakeEnv(is_slippery=False, render_mode="human")
env.spec = gym.spec("FrozenLake-v1")
# display_size = 512
# env.window_size = (display_size, display_size)
# env.cell_size = (
#     env.window_size[0] // env.ncol,
#     env.window_size[1] // env.nrow,
# )
env = gym.wrappers.RecordVideo(env, video_folder="video")

env = DiscreteOneHotWrapper(env)

state, info = env.reset()
total_rewards = 0

while True:
    action = int(torch.argmax(DQN_Q(torch.Tensor(state))))
    state, reward, terminated, truncted, info = env.step(action)
    print(action)
    if terminated:
        break
env.close()

1
1
2
1
2
2
